In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('./train.csv')

In [ ]:
avg_rating_per_user = data.groupby('user')['rating'].mean().reset_index()
avg_rating_per_user.columns = ['user', 'avg_rating']
avg_rating_per_user

In [ ]:
avg_rating_per_user = data.groupby('user')['rating'].agg(['mean', 'count'])
avg_rating_per_user.columns = ['avg_rating', 'count']
avg_rating_per_user

In [ ]:
avg_rating_per_movie = data.groupby('movie')['rating'].agg(['mean', 'count'])
avg_rating_per_movie.columns = ['avg_rating', 'count']
avg_rating_per_movie

In [ ]:
user_avg_rate = avg_rating_per_user.to_dict('index')
user_avg_rate[3952]

In [ ]:
from collections import defaultdict
movie_relative_rate = defaultdict(lambda: {'sum':0, 'count':0})
for row in data.itertuples():
    movie_relative_rate[row[2]]['sum'] += row[4] - user_avg_rate[row[3]]['avg_rating']
    movie_relative_rate[row[2]]['count'] += 1
    
for key, val in movie_relative_rate.items():
    movie_relative_rate[key]['avg'] = val['sum']/val['count']

movie_relative_rate = pd.DataFrame.from_dict(movie_relative_rate, orient='index')
movie_relative_rate